In [ ]:
!pip install -qU pylibjpeg pylibjpeg-openjpeg pylibjpeg-libjpeg pydicom python-gdcm dicomsdl

In [ ]:
import gdcm
import pydicom
import pylibjpeg
import dicomsdl

In [ ]:
from concurrent.futures import ProcessPoolExecutor
import cv2
import glob
import numpy as np
from tqdm import tqdm
import os
import PIL
from PIL import Image
import matplotlib.pyplot as plt
import re
from tqdm.notebook import tqdm
from joblib import Parallel, delayed
import pydicom


In [ ]:
''' Implementation to cut off all the dataset images'''

def cut_off_all(patient, input_path, output_path):
    
    patient_input_path = input_path + '/' + patient
    
    # Create directory if it does not exists
    if not os.path.isdir(output_path+'/'+patient):
        os.makedirs(output_path+'/'+patient)
     
    for img in os.listdir(patient_input_path):
        image_path = patient_input_path + '/'+ img
        
        
        dicom = pydicom.dcmread(image_path)
        
        image = dicom.pixel_array
        image = (image - image.min()) / (image.max() - image.min())
        if dicom.PhotometricInterpretation == "MONOCHROME1":
            image = 1 - image
        #img = cv2.resize(img, (512, 512))
        image = cv2.resize(image, (1024, 912))
        
         
        # 2. Crop
        X = image
        # Some images have narrow exterior "frames" that complicate selection of the main data. Cutting off the frame
        X = X[5:-5, 5:-5]


        # regions of non-empty pixels
        output= cv2.connectedComponentsWithStats((X > 0.05).astype(np.uint8)[:, :], 8, cv2.CV_32S)

        # stats.shape == (N, 5), where N is the number of regions, 5 dimensions correspond to:
        # left, top, width, height, area_size
        stats = output[2]

        # finding max area which always corresponds to the breast data. 
        idx = stats[1:, 4].argmax() + 1
        x1, y1, w, h = stats[idx][:4]
        x2 = x1 + w
        y2 = y1 + h

        # cutting out the breast data
        image_fit = X[y1: y2, x1: x2]
        
        
        #os.makedirs(f'{CFG.test_img_path}/{patient_id}', exist_ok=True)
        #cv2.imwrite(f'{CFG.test_img_path}/{patient_id}/{im_id}.png', (X_fit[:, :] * 255).astype(np.uint8))

        #print(img)
        cv2.imwrite(output_path + '/' + patient + '/' + img.replace('.dcm', '.png'),(image_fit[:, :] * 255).astype(np.uint8))
    

In [ ]:
input_path = '/kaggle/input/rsna-breast-cancer-detection/train_images'
output_path =  '/kaggle/working/output/preprocessed_images_RSNA'

# Parallel loop
_ = Parallel(n_jobs=4)(
    delayed(cut_off_all)(patient, input_path=input_path, output_path=output_path)
    for patient in tqdm(os.listdir(input_path))
)

#patient = '10006'
#cut_off_all(patient, input_path=input_path, output_path=output_path)
    

In [ ]:
# This notebook contains the préprocessing operations copied from vladimir notebook, but putted in my 
# loop (code).
# reference : https://www.kaggle.com/code/rimzakhama/cut-images-from-vladimir/edit